## Gemini 기본 처리

### 라이브러리 설치

In [1]:
%pip install --upgrade --quiet google-genai

### GCP 환경설정 및 로그인

In [2]:
import os

PROJECT_ID = "ai-hangsik" #@param {type:"string"}
REGION = "us-central1" #@param {type:"string"}
USE_VERTEX_AI = True #@param {type:"boolean"}
MODEL = "gemini-2.5-flash" #@param {type:"string"}

In [ ]:
!gcloud auth login
!gcloud config set project {PROJECT_ID}

In [ ]:
!gcloud auth application-default login


### Gemini 실행

In [8]:
import base64
from IPython.display import Image, display, Markdown

from google import genai
from google.genai import types
from google.genai.types import HttpOptions

client = genai.Client(
    vertexai=USE_VERTEX_AI,
    project=PROJECT_ID,
    location=REGION,)

#### Text 입력

In [9]:
from google import genai
from google.genai.types import HttpOptions
from IPython.display import Image, display, Markdown


response = client.models.generate_content(
    model=MODEL,

    contents=[
      types.Content(
        role="user",
        parts=[
          types.Part.from_text(text="""트랜스포머 아키텍처에서 인코더란? """)
        ]
      )
    ],
)
display(Markdown(response.text))

트랜스포머 아키텍처에서 **인코더(Encoder)**는 입력 시퀀스를 처리하고 이해하는 역할을 하는 부분입니다. 주어진 입력 데이터(예: 문장, 텍스트)를 고차원적인 의미론적 표현(contextualized representation)으로 변환하는 역할을 합니다.

### 인코더의 주요 역할 및 기능:

1.  **입력 시퀀스 이해:**
    *   입력 문장 내의 각 단어가 문맥상 어떤 의미를 가지는지 파악합니다.
    *   단순한 단어 임베딩을 넘어, 단어 간의 관계, 의존성 등을 학습하여 풍부한 문맥 정보를 담은 벡터를 생성합니다. 예를 들어, "The animal didn't cross the road because it was too tired." 라는 문장에서 'it'이 'animal'을 가리키는지, 'road'를 가리키는지 등을 파악하는 능력이 여기에 해당합니다.

2.  **문맥화된 벡터 표현 생성:**
    *   인코더의 최종 출력은 입력 시퀀스의 각 단어에 대한 **문맥화된 벡터 표현(contextualized vector representation)**들의 시퀀스입니다. 이 벡터들은 입력 문장의 의미를 깊이 있게 파악한 결과물이며, 디코더에게 전달되어 번역이나 요약 등의 작업을 수행하는 데 활용됩니다.

### 인코더의 내부 구조:

인코더는 여러 개의 동일한 **'인코더 블록(Encoder Block)'**을 쌓아 올린 형태로 구성됩니다. 각 인코더 블록은 크게 두 가지 하위 레이어로 구성됩니다:

1.  **멀티 헤드 셀프-어텐션 레이어 (Multi-Head Self-Attention Layer):**
    *   이 레이어가 인코더의 핵심입니다. 입력 시퀀스 내의 모든 단어가 다른 모든 단어와의 관계를 살펴보며, 어떤 단어에 더 집중해야 할지 학습합니다. 이를 통해 각 단어에 대한 문맥적 의미를 풍부하게 만듭니다.
    *   '멀티 헤드'는 병렬적으로 여러 '헤드'를 사용하여 다양한 관점에서 관계를 파악함으로써, 보다 다각적인 문맥 정보를 추출할 수 있게 합니다.

2.  **포지션-와이즈 피드포워드 네트워크 (Position-wise Feed-Forward Network):**
    *   셀프-어텐션 레이어의 출력을 받아 각 위치(단어)별로 독립적으로 적용되는 완전 연결(Fully Connected) 신경망입니다.
    *   비선형성을 추가하고, 어텐션 레이어에서 학습된 표현을 더 복잡한 형태로 변환하는 역할을 합니다.

각 하위 레이어 뒤에는 **정규화(Add & Normalize)** 및 **잔차 연결(Residual Connection)**이 적용되어 학습 안정성을 높입니다.

### 인코더의 작동 방식:

1.  입력 문장은 먼저 **워드 임베딩(Word Embedding)**과 **위치 임베딩(Positional Encoding)**을 거쳐 인코더의 첫 번째 블록으로 들어갑니다. (위치 임베딩은 순서 정보를 제공합니다.)
2.  이후 각 인코더 블록을 순차적으로 통과하며 셀프-어텐션과 피드포워드 네트워크를 통해 문맥 정보가 축적되고 정제됩니다.
3.  마지막 인코더 블록을 통과한 최종 출력이 디코더로 전달됩니다.

### 비유:

인코더는 마치 한 권의 책을 읽고 그 내용과 각 장의 관계를 완전히 이해하여 핵심 요약본을 만드는 과정과 비슷합니다. 이 요약본(문맥화된 벡터 표현)이 정확해야 다음 단계(디코더의 번역, 요약 등)를 잘 수행할 수 있습니다.

### 인코더의 활용:

트랜스포머 모델이 인코더와 디코더를 모두 사용하는 것은 '시퀀스-투-시퀀스(Seq2Seq)' 작업(예: 기계 번역)일 경우입니다. 하지만 BERT와 같은 모델은 **인코더 부분만을 활용**하여 문장 이해, 분류, 질문 응답 등 다양한 자연어 이해(NLU) 작업에 사용됩니다.

요약하자면, 트랜스포머 인코더는 입력 시퀀스의 모든 단어를 서로 연관 지어 깊이 있게 분석하고, 각 단어의 문맥적 의미를 담은 풍부한 벡터 표현을 생성하는 역할을 하는 '이해 담당자'입니다.

#### Image 입력

In [ ]:
img_file_uri="https://storage.googleapis.com/cloud-samples-data/generative-ai/image/scones.jpg"
display(Image(url=img_file_uri, width=400))

In [ ]:
from google import genai
from google.genai.types import HttpOptions, Part

response = client.models.generate_content(
    model=MODEL,
    contents=[
        Part.from_text(text="""이미지에 보여지는 내용을 설명해주세요"""),
        Part.from_uri(
            file_uri= img_file_uri,
            mime_type="image/jpeg",
        ),
    ],
)
display(Markdown(response.text))

이 이미지는 블루베리 스콘과 신선한 과일, 음료, 꽃 등이 어우러진 아늑하고 먹음직스러운 식탁 풍경을 담고 있습니다.

사진 중앙에는 여러 개의 둥근 블루베리 스콘 또는 비스킷이 하얀 유산지 위에 놓여 있습니다. 스콘은 겉면이 살짝 노릇하고 설탕이 뿌려져 있으며, 속에는 보랏빛 블루베리가 박혀 있어 달콤하고 상큼해 보입니다. 유산지 위에는 블루베리 즙이나 흔적이 군데군데 묻어 있어 자연스러운 느낌을 더합니다.

스콘 주변으로는 다음과 같은 요소들이 배치되어 있습니다:
*   **음료:** 왼쪽 상단에는 진한 색의 커피가 담긴 하얀 머그컵이 보입니다. 오른쪽 하단에는 우유나 크림이 들어간 듯한 연한 갈색 음료가 담긴 또 다른 하얀 잔이 놓여 있습니다.
*   **신선한 블루베리:** 사진 중앙 아래쪽에는 작은 갈색 그릇에 신선한 블루베리가 가득 담겨 있습니다. 스콘 주변 유산지 위에도 몇몇 블루베리가 흩어져 있습니다.
*   **스푼:** 블루베리 그릇 옆에는 "LET'S JAM"이라고 새겨진 은색 스푼이 놓여 있어, 잼과 함께 스콘을 즐기는 분위기를 연출합니다.
*   **꽃:** 사진의 오른편에는 여러 송이의 분홍색 모란(작약) 꽃이 길게 배치되어 있어 화사하고 우아한 분위기를 더합니다.
*   **민트 잎:** 스콘 근처에는 작은 초록색 민트 잎 하나가 놓여 있어 신선함을 강조합니다.

전체적인 배경은 어둡고 질감 있는 표면으로, 따뜻하고 소박하면서도 세련된 느낌을 줍니다. 이 이미지는 여유로운 아침 식사나 오후 티타임을 연상시키며, 신선한 재료와 정성이 느껴지는 디저트의 아름다움을 시각적으로 잘 표현하고 있습니다.

#### Youtube 분석

In [ ]:
from google import genai
from google.genai.types import HttpOptions, Part

response = client.models.generate_content(
    model=MODEL,
    contents=[
        Part.from_uri(
            file_uri="https://www.youtube.com/watch?v=3KtWfp0UopM",
            mime_type="video/mp4",
        ),
        Part.from_text(text="""이 영상을 바탕으로 짧고 매력적인 블로그 게시물을 한국어로 작성해 보세요."""),
    ],
)

display(Markdown(response.text))

## 구글과 함께한 25년: 가장 많이 검색된 이야기들 ✨

25년 전, 구글과 함께 세상은 '검색'이라는 새로운 여정을 시작했습니다. 지난 25년간 구글은 전 세계 수십억 명의 사람들이 궁금해하는 모든 것을 찾아볼 수 있도록 도왔죠. 최근 공개된 구글 25주년 기념 "가장 많이 검색된(The Most Searched)" 비디오는 단순한 검색 기록을 넘어, 우리 시대의 관심사와 역사를 보여주는 감동적인 여정입니다.

영상은 인류의 위대한 첫걸음이었던 **달 착륙**부터 시작해, 가장 많이 검색된 **1980년대**의 문화, 그리고 **포켓몬, 해리포터, 심슨 가족** 같은 추억의 콘텐츠까지 다채로운 과거를 되짚습니다. **BTS**와 **블랙핑크** 같은 K-Pop 아이돌, **비욘세**의 압도적인 공연, **바비** 인형의 진화까지, 검색은 우리가 사랑하고 열광하는 문화 현상의 중심에 늘 함께했습니다.

스포츠 스타 **호날두**와 **르브론 제임스**, 크리켓 선수 **비라트 콜리**를 통해 최고들의 열정을 보여주고, 스파이더맨 같은 슈퍼히어로뿐만 아니라 소방관, 의료진, 재난 구조대원 같은 현실 영웅들의 숭고한 정신을 조명합니다. 핵융합과 같은 과학적 돌파구, 그리고 블랙 라이브스 매터처럼 중요한 사회 운동까지, 검색은 우리 사회의 발전과 인류의 노력을 기록해왔습니다.

특히 '도움이 필요한 사람 돕는 법'과 같은 검색어는 우리가 가진 선한 의지를 보여줍니다. 아이스버킷 챌린지 같은 바이럴 캠페인, 그리고 프라이드 퍼레이드처럼 다양성을 존중하는 움직임들은 검색이 단순한 정보 전달을 넘어, 공동체 의식을 형성하고 긍정적인 변화를 이끌어냈음을 증명하죠.

영상 마지막에 등장하는 "기준을 세우는 사람들, 그리고 그 기준을 더 높이 올릴 사람들"이라는 메시지는 미래 세대에 대한 희망과 기대를 담고 있습니다. 어린 아이들이 농구, 축구, 역도, 스케이트보드 등 다양한 분야에서 잠재력을 뽐내는 모습은 검색이 단순히 과거를 보여주는 것을 넘어, 미래를 향한 무한한 가능성을 열어준다는 것을 상기시킵니다.

지난 25년간 구글 검색은 우리 삶의 거울이자, 지식과 영감의 원천이었습니다. 과거를 기억하고, 현재를 이해하며, 더 나은 미래를 꿈꾸게 하는 힘. 이제 다음 25년을 향해, 계속해서 '검색하세요(Search on).'

구글의 25주년을 축하합니다! 🎉

## End of Document